In [ ]:
import torch

from pytorch_forecasting import DeepAR

checkpoint = torch.load('epoch=1-step=60.ckpt')




print(checkpoint['optimizer_states'])
best_tft = DeepAR.load_from_checkpoint(best_model_path)


In [ ]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pytorch_forecasting.data import TimeSeriesDataSet
from pytorch_forecasting.data import NaNLabelEncoder
from pytorch_forecasting.data.encoders import TorchNormalizer
from pytorch_forecasting import Baseline
import torch
from pytorch_forecasting.metrics import SMAPE

df = pd.DataFrame({"cat":np.array(['rain','shine','drizzle','hail','storm',"shine","rain","hail","drizzle"])})
df["time_idx"] = np.arange(0,len(df))
df["group"] = np.tile(0,len(df))
df["value"] = np.random.rand(len(df))
df["temp"] = np.arange(325,325+len(df))
Target = "value"


df["date"] = pd.Timestamp("2021-09-24") + pd.to_timedelta(df.time_idx, "H")
df['_hour_of_day'] = df["date"].dt.hour.astype(str)
df['_day_of_week'] = df["date"].dt.dayofweek.astype(str)
df['_day_of_month'] = df["date"].dt.day.astype(str)
df['_day_of_year'] = df["date"].dt.dayofyear.astype(str)
df['_week_of_year'] = df["date"].dt.weekofyear.astype(str)
df['_month_of_year'] = df["date"].dt.month.astype(str)
df['_year'] = df["date"].dt.year.astype(str)

max_encoder_length = 3
max_prediction_length = 2

train_dataset = TimeSeriesDataSet(
    df.iloc[0:len(df)],
    time_idx="time_idx",
    target=Target,
    categorical_encoders={"cat": NaNLabelEncoder(add_nan=True).fit(df.cat), "_hour_of_day": NaNLabelEncoder(add_nan=True).fit(df._hour_of_day), \
       "_day_of_week": NaNLabelEncoder(add_nan=True).fit(df._day_of_week), "_day_of_month" : NaNLabelEncoder(add_nan=True).fit(df._day_of_month), "_day_of_year" : NaNLabelEncoder(add_nan=True).fit(df._day_of_year), \
        "_week_of_year": NaNLabelEncoder(add_nan=True).fit(df._week_of_year), "_month_of_year": NaNLabelEncoder(add_nan=True).fit(df._month_of_year) ,"_year": NaNLabelEncoder(add_nan=True).fit(df._year)},
    group_ids=["group"],
    min_encoder_length=max_encoder_length,
    max_encoder_length=max_encoder_length,
    min_prediction_length=max_prediction_length,
    max_prediction_length=max_prediction_length,
    time_varying_unknown_categoricals=["cat","_hour_of_day","_day_of_week","_day_of_month","_day_of_year","_week_of_year","_month_of_year","_year"],
    time_varying_unknown_reals=[Target,"temp"],
    add_relative_time_idx=False,
    randomize_length=None,
    scalers={},
    target_normalizer=TorchNormalizer(method="identity",center=False,transformation=None )

)

batch_size = 2
train_dataloader = train_dataset.to_dataloader(train=True, batch_size=batch_size, num_workers=0)

print(df)
print(train_dataset.data)



In [55]:

# calculate baseline absolute error
actuals = torch.cat([y[0] for x, y in iter(train_dataloader)])
for x, y in iter(train_dataloader):
    print(x)
baseline_predictions = Baseline().predict(train_dataloader)


print(baseline_predictions)

SMAPE()(baseline_predictions, actuals)


{'encoder_cat': tensor([[[3, 1, 1, 1, 1, 1, 1, 1],
         [4, 2, 1, 1, 1, 1, 1, 1],
         [1, 3, 1, 1, 1, 1, 1, 1]],

        [[1, 3, 1, 1, 1, 1, 1, 1],
         [2, 4, 1, 1, 1, 1, 1, 1],
         [5, 5, 1, 1, 1, 1, 1, 1]]]), 'encoder_cont': tensor([[[ 0.1738, -1.5492],
         [ 0.8419, -1.1619],
         [ 0.7271, -0.7746]],

        [[ 0.7271, -0.7746],
         [ 0.6414, -0.3873],
         [ 0.0627,  0.0000]]]), 'encoder_target': tensor([[0.1738, 0.8419, 0.7271],
        [0.7271, 0.6414, 0.0627]]), 'encoder_lengths': tensor([3, 3]), 'decoder_cat': tensor([[[2, 4, 1, 1, 1, 1, 1, 1],
         [5, 5, 1, 1, 1, 1, 1, 1]],

        [[4, 6, 1, 1, 1, 1, 1, 1],
         [3, 7, 1, 1, 1, 1, 1, 1]]]), 'decoder_cont': tensor([[[ 0.6414, -0.3873],
         [ 0.0627,  0.0000]],

        [[ 0.7052,  0.3873],
         [ 0.3445,  0.7746]]]), 'decoder_target': tensor([[0.6414, 0.0627],
        [0.7052, 0.3445]]), 'decoder_lengths': tensor([2, 2]), 'decoder_time_idx': tensor([[3, 4],
        [5,